# 🧲 Levitador Magnético Benchmark Tutorial

## Comprehensive Guide with Visualizations

**Author:** José de Jesús Santana Ramírez  
**Institution:** Universidad Autónoma de Querétaro  
**Version:** 1.0  
**Date:** December 2024

---

## 📚 Table of Contents

1. [Introduction to the Physical System](#section-1)
2. [Understanding the Benchmark](#section-2)
3. [Loading and Visualizing Data](#section-3)
4. [Parameter Space Exploration](#section-4)
5. [Fitness Landscape Visualization](#section-5)
6. [Comparing Different Solutions](#section-6)
7. [Running Optimization Examples](#section-7)
8. [Analyzing Results](#section-8)

---

<a id='section-1'></a>
## 1. 🎯 Introduction to the Physical System

### What is a Magnetic Levitator?

The magnetic levitator is a physical system where a steel sphere is suspended by an electromagnet. The system is inherently **unstable** and requires active control to maintain the sphere at a desired position.

### The Mathematical Model

The inductance of the electromagnet varies with the distance to the sphere:

$$L(y) = k_0 + \frac{k}{1 + y/a}$$

Where:
- $k_0$ : Base inductance [H]
- $k$ : Inductance coefficient [H]
- $a$ : Geometric parameter [m]
- $y$ : Sphere position [m]

### The Optimization Problem

**Goal:** Find the parameters $[k_0, k, a]$ that minimize the Mean Squared Error (MSE) between:
- Simulated trajectory from the digital twin model
- Real experimental data from the physical system

In [ ]:
# Import necessary libraries
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from levitador_benchmark import LevitadorBenchmark
import warnings
# Filter specific warnings that are expected in numerical optimization
warnings.filterwarnings('ignore', category=FutureWarning)
warnings.filterwarnings('ignore', category=UserWarning, module='matplotlib')

# Configure plotting style
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")
%matplotlib inline

# Set random seed for reproducibility
np.random.seed(42)

print("✓ Libraries imported successfully")
print(f"NumPy version: {np.__version__}")

<a id='section-2'></a>
## 2. 🔍 Understanding the Benchmark

The `LevitadorBenchmark` class provides a standardized interface for testing optimization algorithms on a real-world problem.

In [ ]:
# Create the benchmark instance
# Using real experimental data if available, otherwise synthetic data
try:
    benchmark = LevitadorBenchmark("data/datos_levitador.txt", random_seed=42)
    print("✓ Using real experimental data")
except:
    benchmark = LevitadorBenchmark(random_seed=42)
    print("✓ Using synthetic data")

print("\n" + "="*60)
print("BENCHMARK PROPERTIES")
print("="*60)
print(f"Problem dimension: {benchmark.dim}")
print(f"Number of data points: {len(benchmark.t_real)}")
print(f"Time span: {benchmark.t_real[0]:.3f} - {benchmark.t_real[-1]:.3f} seconds")
print(f"\nParameter bounds:")
for i, (name, (lb, ub)) in enumerate(zip(benchmark.variable_names, benchmark.bounds)):
    print(f"  {name:3s}: [{lb:.4f}, {ub:.4f}]")
print(f"\nReference solution: {benchmark.reference_solution}")

<a id='section-3'></a>
## 3. 📊 Loading and Visualizing Data

Let's visualize the experimental data that serves as ground truth for our optimization.

In [ ]:
# Visualize the experimental data
fig, axes = plt.subplots(2, 2, figsize=(14, 8))
fig.suptitle('Experimental Data from Magnetic Levitator', fontsize=16, fontweight='bold')

# Position vs Time
axes[0, 0].plot(benchmark.t_real * 1000, benchmark.y_real * 1000, 'b-', linewidth=1.5)
axes[0, 0].set_xlabel('Time [ms]', fontsize=11)
axes[0, 0].set_ylabel('Position [mm]', fontsize=11)
axes[0, 0].set_title('Sphere Position vs Time', fontsize=12, fontweight='bold')
axes[0, 0].grid(True, alpha=0.3)

# Current vs Time
axes[0, 1].plot(benchmark.t_real * 1000, benchmark.i_real, 'r-', linewidth=1.5)
axes[0, 1].set_xlabel('Time [ms]', fontsize=11)
axes[0, 1].set_ylabel('Current [A]', fontsize=11)
axes[0, 1].set_title('Electromagnet Current vs Time', fontsize=12, fontweight='bold')
axes[0, 1].grid(True, alpha=0.3)

# Voltage vs Time
axes[1, 0].plot(benchmark.t_real * 1000, benchmark.u_real, 'g-', linewidth=1.5)
axes[1, 0].set_xlabel('Time [ms]', fontsize=11)
axes[1, 0].set_ylabel('Voltage [V]', fontsize=11)
axes[1, 0].set_title('Input Voltage vs Time', fontsize=12, fontweight='bold')
axes[1, 0].grid(True, alpha=0.3)

# Phase portrait: Position vs Current
axes[1, 1].scatter(benchmark.y_real * 1000, benchmark.i_real, c=benchmark.t_real, 
                   cmap='viridis', s=10, alpha=0.6)
axes[1, 1].set_xlabel('Position [mm]', fontsize=11)
axes[1, 1].set_ylabel('Current [A]', fontsize=11)
axes[1, 1].set_title('Phase Portrait: Position vs Current', fontsize=12, fontweight='bold')
axes[1, 1].grid(True, alpha=0.3)
cbar = plt.colorbar(axes[1, 1].collections[0], ax=axes[1, 1])
cbar.set_label('Time [s]', fontsize=10)

plt.tight_layout()
plt.show()

print("\n📊 Data Statistics:")
print(f"Position range: [{benchmark.y_real.min()*1000:.2f}, {benchmark.y_real.max()*1000:.2f}] mm")
print(f"Current range: [{benchmark.i_real.min():.2f}, {benchmark.i_real.max():.2f}] A")
print(f"Voltage range: [{benchmark.u_real.min():.2f}, {benchmark.u_real.max():.2f}] V")

<a id='section-4'></a>
## 4. 🔬 Parameter Space Exploration

Let's explore how different parameter values affect the fitness function.

In [ ]:
# Evaluate random solutions in the parameter space
print("Evaluating random solutions in parameter space...")
n_samples = 200
solutions = []
errors = []

np.random.seed(42)
for i in range(n_samples):
    # Generate random solution within bounds
    solution = [np.random.uniform(lb, ub) for lb, ub in benchmark.bounds]
    error = benchmark.fitness_function(solution)
    solutions.append(solution)
    errors.append(error)
    
    if (i + 1) % 50 == 0:
        print(f"  Evaluated {i+1}/{n_samples} solutions")

solutions = np.array(solutions)
errors = np.array(errors)

# Filter out extreme errors for better visualization
valid_mask = errors < np.percentile(errors, 95)
solutions_filtered = solutions[valid_mask]
errors_filtered = errors[valid_mask]

print(f"\n✓ Completed {n_samples} evaluations")
print(f"Best error found: {errors.min():.6e}")
print(f"Worst error found: {errors.max():.6e}")
print(f"Mean error: {errors.mean():.6e}")

In [ ]:
# Visualize parameter space exploration
fig = plt.figure(figsize=(16, 10))
fig.suptitle('Parameter Space Exploration', fontsize=16, fontweight='bold')

# Create 3D scatter plot
ax1 = fig.add_subplot(2, 3, 1, projection='3d')
scatter = ax1.scatter(solutions_filtered[:, 0], solutions_filtered[:, 1], 
                      solutions_filtered[:, 2], c=np.log10(errors_filtered), 
                      cmap='jet', s=50, alpha=0.6)
ax1.set_xlabel('k0 [H]', fontsize=10)
ax1.set_ylabel('k [H]', fontsize=10)
ax1.set_zlabel('a [m]', fontsize=10)
ax1.set_title('3D Parameter Space\n(color = log10(error))', fontsize=11, fontweight='bold')
plt.colorbar(scatter, ax=ax1, label='log10(MSE)', shrink=0.7)

# 2D projections with error as color
param_names = ['k0 [H]', 'k [H]', 'a [m]']
projections = [(0, 1), (0, 2), (1, 2)]

for idx, (i, j) in enumerate(projections, start=2):
    ax = fig.add_subplot(2, 3, idx)
    scatter = ax.scatter(solutions_filtered[:, i], solutions_filtered[:, j], 
                        c=np.log10(errors_filtered), cmap='jet', s=30, alpha=0.6)
    ax.set_xlabel(param_names[i], fontsize=10)
    ax.set_ylabel(param_names[j], fontsize=10)
    ax.set_title(f'{param_names[i]} vs {param_names[j]}', fontsize=11, fontweight='bold')
    ax.grid(True, alpha=0.3)
    plt.colorbar(scatter, ax=ax, label='log10(MSE)')

# Distribution of errors
ax = fig.add_subplot(2, 3, 5)
ax.hist(np.log10(errors_filtered), bins=30, color='steelblue', edgecolor='black', alpha=0.7)
ax.set_xlabel('log10(MSE)', fontsize=11)
ax.set_ylabel('Frequency', fontsize=11)
ax.set_title('Distribution of Fitness Errors', fontsize=11, fontweight='bold')
ax.grid(True, alpha=0.3)
ax.axvline(np.log10(errors.min()), color='red', linestyle='--', linewidth=2, label='Best')
ax.legend()

# Box plots for parameter distributions
ax = fig.add_subplot(2, 3, 6)
box_data = [solutions_filtered[:, i] for i in range(3)]
bp = ax.boxplot(box_data, labels=['k0', 'k', 'a'], patch_artist=True)
for patch, color in zip(bp['boxes'], ['lightblue', 'lightgreen', 'lightcoral']):
    patch.set_facecolor(color)
ax.set_ylabel('Parameter Value', fontsize=11)
ax.set_title('Parameter Value Distributions', fontsize=11, fontweight='bold')
ax.grid(True, alpha=0.3, axis='y')

plt.tight_layout()
plt.show()

<a id='section-5'></a>
## 5. 🗺️ Fitness Landscape Visualization

Let's visualize 2D slices of the fitness landscape to understand the optimization challenge.

In [ ]:
# Create 2D fitness landscape slices
print("Creating fitness landscape visualizations...")
print("This may take a few minutes...")

# Use reference solution as center point
ref_sol = benchmark.reference_solution

# Define grid resolution
n_points = 30

# Create landscapes for each pair of parameters
landscapes = []

# k0 vs k (fixing a at reference)
k0_range = np.linspace(benchmark.bounds[0][0], benchmark.bounds[0][1], n_points)
k_range = np.linspace(benchmark.bounds[1][0], benchmark.bounds[1][1], n_points)
K0, K = np.meshgrid(k0_range, k_range)
Z1 = np.zeros_like(K0)

print("  Computing k0-k landscape...")
for i in range(n_points):
    for j in range(n_points):
        Z1[i, j] = benchmark.fitness_function([K0[i, j], K[i, j], ref_sol[2]])
landscapes.append((K0, K, Z1, 'k0', 'k'))

# k0 vs a (fixing k at reference)
a_range = np.linspace(benchmark.bounds[2][0], benchmark.bounds[2][1], n_points)
K0_2, A = np.meshgrid(k0_range, a_range)
Z2 = np.zeros_like(K0_2)

print("  Computing k0-a landscape...")
for i in range(n_points):
    for j in range(n_points):
        Z2[i, j] = benchmark.fitness_function([K0_2[i, j], ref_sol[1], A[i, j]])
landscapes.append((K0_2, A, Z2, 'k0', 'a'))

# k vs a (fixing k0 at reference)
K_2, A_2 = np.meshgrid(k_range, a_range)
Z3 = np.zeros_like(K_2)

print("  Computing k-a landscape...")
for i in range(n_points):
    for j in range(n_points):
        Z3[i, j] = benchmark.fitness_function([ref_sol[0], K_2[i, j], A_2[i, j]])
landscapes.append((K_2, A_2, Z3, 'k', 'a'))

print("✓ Landscapes computed")

In [ ]:
# Visualize fitness landscapes
fig, axes = plt.subplots(2, 3, figsize=(16, 10))
fig.suptitle('Fitness Landscape Slices (fixing one parameter at reference value)', 
             fontsize=16, fontweight='bold')

# 3D surface plots
for idx, (X, Y, Z, xlabel, ylabel) in enumerate(landscapes):
    ax = fig.add_subplot(2, 3, idx + 1, projection='3d')
    
    # Clip extreme values for better visualization
    Z_clipped = np.clip(Z, 0, np.percentile(Z, 95))
    
    surf = ax.plot_surface(X, Y, np.log10(Z_clipped + 1e-10), cmap='viridis', 
                           alpha=0.9, edgecolor='none')
    ax.set_xlabel(f'{xlabel} [H]' if xlabel != 'a' else f'{xlabel} [m]', fontsize=10)
    ax.set_ylabel(f'{ylabel} [H]' if ylabel != 'a' else f'{ylabel} [m]', fontsize=10)
    ax.set_zlabel('log10(MSE)', fontsize=10)
    ax.set_title(f'{xlabel} vs {ylabel}', fontsize=11, fontweight='bold')
    fig.colorbar(surf, ax=ax, shrink=0.5, label='log10(MSE)')

# Contour plots
for idx, (X, Y, Z, xlabel, ylabel) in enumerate(landscapes):
    ax = fig.add_subplot(2, 3, idx + 4)
    
    Z_clipped = np.clip(Z, 0, np.percentile(Z, 95))
    
    contour = ax.contourf(X, Y, np.log10(Z_clipped + 1e-10), levels=20, cmap='viridis')
    ax.contour(X, Y, np.log10(Z_clipped + 1e-10), levels=20, colors='black', 
               alpha=0.3, linewidths=0.5)
    
    # Mark reference solution
    ref_idx_map = {'k0': 0, 'k': 1, 'a': 2}
    ref_x = ref_sol[ref_idx_map[xlabel]]
    ref_y = ref_sol[ref_idx_map[ylabel]]
    ax.plot(ref_x, ref_y, 'r*', markersize=15, label='Reference', markeredgecolor='white', markeredgewidth=1)
    
    ax.set_xlabel(f'{xlabel} [H]' if xlabel != 'a' else f'{xlabel} [m]', fontsize=10)
    ax.set_ylabel(f'{ylabel} [H]' if ylabel != 'a' else f'{ylabel} [m]', fontsize=10)
    ax.set_title(f'Contour: {xlabel} vs {ylabel}', fontsize=11, fontweight='bold')
    ax.legend(loc='upper right', fontsize=8)
    fig.colorbar(contour, ax=ax, label='log10(MSE)')

plt.tight_layout()
plt.show()

print("\n💡 Observation:")
print("The fitness landscape shows the complexity of this optimization problem.")
print("Notice the valleys and ridges that make this a challenging problem for algorithms.")

<a id='section-6'></a>
## 6. 🔄 Comparing Different Solutions

Let's compare how different parameter sets perform by visualizing their simulation results.

In [ ]:
# Define different solutions to compare
solutions_to_compare = {
    'Reference': benchmark.reference_solution,
    'Random 1': [np.random.uniform(lb, ub) for lb, ub in benchmark.bounds],
    'Random 2': [np.random.uniform(lb, ub) for lb, ub in benchmark.bounds],
    'Upper bounds': [ub for lb, ub in benchmark.bounds],
    'Lower bounds': [lb for lb, ub in benchmark.bounds],
    'Midpoint': [(lb + ub) / 2 for lb, ub in benchmark.bounds]
}

# Evaluate each solution
print("Comparing different solutions:\n")
print("="*80)
print(f"{'Solution Name':<20} {'k0':>10} {'k':>10} {'a':>10} {'MSE':>15}")
print("="*80)

solution_errors = {}
for name, sol in solutions_to_compare.items():
    error = benchmark.fitness_function(sol)
    solution_errors[name] = error
    print(f"{name:<20} {sol[0]:>10.5f} {sol[1]:>10.5f} {sol[2]:>10.5f} {error:>15.6e}")

print("="*80)

In [ ]:
# Visualize comparison of solutions
from scipy.integrate import odeint

fig, axes = plt.subplots(2, 3, figsize=(16, 10))
fig.suptitle('Comparison of Different Parameter Solutions', fontsize=16, fontweight='bold')

# Plot ground truth in all subplots
for idx, (name, sol) in enumerate(list(solutions_to_compare.items())[:6]):
    ax = axes[idx // 3, idx % 3]
    
    # Plot experimental data
    ax.plot(benchmark.t_real * 1000, benchmark.y_real * 1000, 'b-', 
            linewidth=2, label='Experimental', alpha=0.7)
    
    # Simulate with these parameters
    k0, k, a = sol
    y0 = benchmark.y_real[0]
    try:
        solution = odeint(benchmark._modelo_dinamico, [y0, 0, 0], 
                         benchmark.t_real, args=(k0, k, a))
        y_sim = solution[:, 0]
        
        ax.plot(benchmark.t_real * 1000, y_sim * 1000, 'r--', 
                linewidth=2, label='Simulation', alpha=0.7)
        
        error = solution_errors[name]
        ax.set_title(f'{name}\nMSE: {error:.3e}', fontsize=11, fontweight='bold')
    except:
        ax.set_title(f'{name}\n(Simulation failed)', fontsize=11, fontweight='bold', color='red')
    
    ax.set_xlabel('Time [ms]', fontsize=10)
    ax.set_ylabel('Position [mm]', fontsize=10)
    ax.legend(loc='best', fontsize=8)
    ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Bar plot of errors
fig, ax = plt.subplots(figsize=(12, 6))
names = list(solution_errors.keys())
errors_list = [solution_errors[name] for name in names]

# Use log scale for better visualization
colors = ['green' if name == 'Reference' else 'steelblue' for name in names]
bars = ax.bar(names, np.log10(np.array(errors_list) + 1e-12), color=colors, edgecolor='black', alpha=0.7)
ax.set_ylabel('log10(MSE)', fontsize=12)
ax.set_title('Comparison of Solution Quality (lower is better)', fontsize=14, fontweight='bold')
ax.grid(True, alpha=0.3, axis='y')
plt.xticks(rotation=45, ha='right')

# Add value labels on bars
for i, (bar, error) in enumerate(zip(bars, errors_list)):
    height = bar.get_height()
    ax.text(bar.get_x() + bar.get_width()/2., height,
            f'{error:.2e}',
            ha='center', va='bottom', fontsize=9, rotation=0)

plt.tight_layout()
plt.show()

<a id='section-7'></a>
## 7. 🚀 Running Optimization Examples

Now let's run some optimization algorithms on the benchmark and visualize their convergence.

In [ ]:
# Example 1: Random Search (Baseline)
print("Running Random Search...")

n_iterations = 100
best_error = float('inf')
best_solution = None
error_history_random = []

np.random.seed(42)
for i in range(n_iterations):
    solution = [np.random.uniform(lb, ub) for lb, ub in benchmark.bounds]
    error = benchmark.fitness_function(solution)
    
    if error < best_error:
        best_error = error
        best_solution = solution
    
    error_history_random.append(best_error)

print(f"\n✓ Random Search completed")
print(f"Best solution: [{best_solution[0]:.5f}, {best_solution[1]:.5f}, {best_solution[2]:.5f}]")
print(f"Best error: {best_error:.6e}")

In [ ]:
# Example 2: Differential Evolution (SciPy)
print("Running Differential Evolution...")

from scipy.optimize import differential_evolution

# Track convergence
error_history_de = []

def callback_de(xk, convergence):
    error = benchmark.fitness_function(xk)
    error_history_de.append(error)

result = differential_evolution(
    benchmark.fitness_function,
    benchmark.bounds,
    strategy='best1bin',
    maxiter=50,
    popsize=15,
    seed=42,
    callback=callback_de,
    disp=False
)

print(f"\n✓ Differential Evolution completed")
print(f"Best solution: [{result.x[0]:.5f}, {result.x[1]:.5f}, {result.x[2]:.5f}]")
print(f"Best error: {result.fun:.6e}")
print(f"Evaluations: {result.nfev}")

In [ ]:
# Visualize convergence comparison
fig, axes = plt.subplots(1, 2, figsize=(14, 5))
fig.suptitle('Optimization Algorithm Convergence Comparison', fontsize=16, fontweight='bold')

# Linear scale
axes[0].plot(error_history_random, 'b-', linewidth=2, label='Random Search', alpha=0.7)
axes[0].plot(error_history_de, 'r-', linewidth=2, label='Differential Evolution', alpha=0.7)
axes[0].axhline(y=benchmark.fitness_function(benchmark.reference_solution), 
                color='g', linestyle='--', linewidth=2, label='Reference Solution', alpha=0.7)
axes[0].set_xlabel('Iteration', fontsize=12)
axes[0].set_ylabel('Best MSE', fontsize=12)
axes[0].set_title('Linear Scale', fontsize=12, fontweight='bold')
axes[0].legend(fontsize=10)
axes[0].grid(True, alpha=0.3)

# Log scale
axes[1].semilogy(error_history_random, 'b-', linewidth=2, label='Random Search', alpha=0.7)
axes[1].semilogy(error_history_de, 'r-', linewidth=2, label='Differential Evolution', alpha=0.7)
axes[1].axhline(y=benchmark.fitness_function(benchmark.reference_solution), 
                color='g', linestyle='--', linewidth=2, label='Reference Solution', alpha=0.7)
axes[1].set_xlabel('Iteration', fontsize=12)
axes[1].set_ylabel('Best MSE (log scale)', fontsize=12)
axes[1].set_title('Logarithmic Scale', fontsize=12, fontweight='bold')
axes[1].legend(fontsize=10)
axes[1].grid(True, alpha=0.3, which='both')

plt.tight_layout()
plt.show()

print("\n📊 Performance Summary:")
print(f"Random Search - Final Error: {error_history_random[-1]:.6e}")
print(f"Differential Evolution - Final Error: {error_history_de[-1]:.6e}")
improvement = (error_history_random[-1] - error_history_de[-1]) / error_history_random[-1] * 100
print(f"\nDE improvement over Random Search: {improvement:.2f}%")

<a id='section-8'></a>
## 8. 📈 Analyzing Results

Let's analyze the best solution found and compare it visually with the reference.

In [ ]:
# Use benchmark's built-in visualization
print("Visualizing the best solution from Differential Evolution:")
benchmark.visualize_solution(result.x)

print("\nVisualizing the reference solution:")
benchmark.visualize_solution(benchmark.reference_solution)

In [ ]:
# Detailed comparison: DE solution vs Reference
fig, axes = plt.subplots(2, 2, figsize=(14, 10))
fig.suptitle('Detailed Analysis: Best Found Solution vs Reference', fontsize=16, fontweight='bold')

# Simulate both solutions
k0_de, k_de, a_de = result.x
k0_ref, k_ref, a_ref = benchmark.reference_solution
y0 = benchmark.y_real[0]

sol_de = odeint(benchmark._modelo_dinamico, [y0, 0, 0], benchmark.t_real, args=(k0_de, k_de, a_de))
sol_ref = odeint(benchmark._modelo_dinamico, [y0, 0, 0], benchmark.t_real, args=(k0_ref, k_ref, a_ref))

y_sim_de = sol_de[:, 0]
y_sim_ref = sol_ref[:, 0]

# Position comparison
axes[0, 0].plot(benchmark.t_real * 1000, benchmark.y_real * 1000, 'k-', 
                linewidth=2, label='Experimental', alpha=0.8)
axes[0, 0].plot(benchmark.t_real * 1000, y_sim_de * 1000, 'r--', 
                linewidth=2, label='DE Solution', alpha=0.7)
axes[0, 0].plot(benchmark.t_real * 1000, y_sim_ref * 1000, 'g:', 
                linewidth=2, label='Reference', alpha=0.7)
axes[0, 0].set_xlabel('Time [ms]', fontsize=11)
axes[0, 0].set_ylabel('Position [mm]', fontsize=11)
axes[0, 0].set_title('Position Trajectories', fontsize=12, fontweight='bold')
axes[0, 0].legend(fontsize=10)
axes[0, 0].grid(True, alpha=0.3)

# Error over time for DE solution
error_de = (benchmark.y_real - y_sim_de) * 1000  # in mm
error_ref = (benchmark.y_real - y_sim_ref) * 1000

axes[0, 1].plot(benchmark.t_real * 1000, error_de, 'r-', linewidth=1.5, label='DE Error', alpha=0.7)
axes[0, 1].plot(benchmark.t_real * 1000, error_ref, 'g-', linewidth=1.5, label='Reference Error', alpha=0.7)
axes[0, 1].axhline(y=0, color='k', linestyle='--', linewidth=1, alpha=0.5)
axes[0, 1].set_xlabel('Time [ms]', fontsize=11)
axes[0, 1].set_ylabel('Error [mm]', fontsize=11)
axes[0, 1].set_title('Tracking Error Over Time', fontsize=12, fontweight='bold')
axes[0, 1].legend(fontsize=10)
axes[0, 1].grid(True, alpha=0.3)

# Parameter comparison bar chart
param_names = ['k0', 'k', 'a']
x_pos = np.arange(len(param_names))
width = 0.35

axes[1, 0].bar(x_pos - width/2, result.x, width, label='DE Solution', color='red', alpha=0.7)
axes[1, 0].bar(x_pos + width/2, benchmark.reference_solution, width, label='Reference', color='green', alpha=0.7)
axes[1, 0].set_xlabel('Parameters', fontsize=11)
axes[1, 0].set_ylabel('Value', fontsize=11)
axes[1, 0].set_title('Parameter Values Comparison', fontsize=12, fontweight='bold')
axes[1, 0].set_xticks(x_pos)
axes[1, 0].set_xticklabels(param_names)
axes[1, 0].legend(fontsize=10)
axes[1, 0].grid(True, alpha=0.3, axis='y')

# Error statistics
stats_data = [
    ['Metric', 'DE Solution', 'Reference'],
    ['MSE', f'{result.fun:.6e}', f'{benchmark.fitness_function(benchmark.reference_solution):.6e}'],
    ['MAE [mm]', f'{np.mean(np.abs(error_de)):.4f}', f'{np.mean(np.abs(error_ref)):.4f}'],
    ['Max Error [mm]', f'{np.max(np.abs(error_de)):.4f}', f'{np.max(np.abs(error_ref)):.4f}'],
    ['Std Error [mm]', f'{np.std(error_de):.4f}', f'{np.std(error_ref):.4f}']
]

axes[1, 1].axis('tight')
axes[1, 1].axis('off')
table = axes[1, 1].table(cellText=stats_data, cellLoc='center', loc='center',
                         colWidths=[0.3, 0.35, 0.35])
table.auto_set_font_size(False)
table.set_fontsize(10)
table.scale(1, 2)

# Style header row
for i in range(3):
    cell = table[(0, i)]
    cell.set_facecolor('#4CAF50')
    cell.set_text_props(weight='bold', color='white')

axes[1, 1].set_title('Error Statistics', fontsize=12, fontweight='bold', pad=20)

plt.tight_layout()
plt.show()

print("\n📊 Final Analysis:")
print(f"DE Solution MSE: {result.fun:.6e}")
print(f"Reference MSE: {benchmark.fitness_function(benchmark.reference_solution):.6e}")
print(f"\nDE Solution Parameters: k0={k0_de:.5f}, k={k_de:.5f}, a={a_de:.5f}")
print(f"Reference Parameters: k0={k0_ref:.5f}, k={k_ref:.5f}, a={a_ref:.5f}")

## 🎓 Summary and Next Steps

### What We've Learned

1. **Physical System**: The magnetic levitator is a real-world system with complex nonlinear dynamics
2. **Benchmark Interface**: Simple to use with `fitness_function()` for any optimization algorithm
3. **Challenge**: The fitness landscape is complex with multiple local optima
4. **Visualization**: Critical for understanding algorithm behavior and solution quality

### Next Steps

1. **Try Different Algorithms**: Test PSO, Genetic Algorithms, Grey Wolf, etc.
2. **Tune Hyperparameters**: Experiment with population size, mutation rates, etc.
3. **Compare Performance**: Run multiple trials and compute statistics
4. **Use Real Data**: Load your own experimental data from the physical system

### Resources

- **Repository**: [github.com/JRavenelco/levitador-benchmark](https://github.com/JRavenelco/levitador-benchmark)
- **Documentation**: See README.md for detailed information
- **Examples**: Check `example_optimization.py` for more algorithm implementations
- **Metaheuristics Tutorial**: See `tutorial_metaheuristicas.ipynb` for implementing your own algorithms

---

**Happy Optimizing! 🚀**